Install Dependencies

In [ ]:
!pip install accelerate -U

In [ ]:
! pip install datasets transformers

Generate HuggingFace Tokens

In [ ]:
from huggingface_hub import notebook_login
#huggingface token.
notebook_login()

Then you need to install Git-LFS.

In [ ]:
 !apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

4.38.2


We also quickly upload some telemetry - this tells us which examples and

1.   List item
2.   List item

software versions are getting used so we know where to prioritize our maintenance efforts.

In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("language_modeling", framework="pytorch")

# Fine-tuning a language model

 how to fine-tune one of the [ Transformers](https://github.com/huggingface/transformers) model on a language modeling tasks. We will cover two types of language modeling tasks which are:

- Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.

![Widget inference representing the causal language modeling task](https://github.com/huggingface/notebooks/blob/main/examples/images/causal_language_modeling.png?raw=1)

- Masked language modeling: the model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value.

![Widget inference representing the masked language modeling task](https://github.com/huggingface/notebooks/blob/main/examples/images/masked_language_modeling.png?raw=1)

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `Trainer` API to fine-tune a model on it.



## Preparing the dataset

For each of those tasks, we will use the [Wikitext 2]() dataset as an example.

In [ ]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [ ]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

In [ ]:
type(datasets)

datasets.dataset_dict.DatasetDict

In [ ]:
datasets.keys()

dict_keys(['test', 'train', 'validation'])

In [ ]:
datasets["train"].to_csv("train.csv")
datasets["validation"].to_csv("validation.csv" )
datasets['test'].to_csv( "test.csv" )

Creating CSV from Arrow format:   0%|          | 0/37 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

1303084

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
print(datasets['train'].shape)
print(datasets['validation'].shape)
print(datasets['test'].shape)

(36718, 1)
(3760, 1)
(4358, 1)


In [ ]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,text
0,"Politically , Ireland is divided between the Republic of Ireland ( officially named Ireland ) , which covers five @-@ sixths of the island , and Northern Ireland , which is part of the United Kingdom , in the northeast of the island . In 2011 the population of Ireland was about 6 @.@ 4 million , ranking it the second @-@ most populous island in Europe after Great Britain . Just under 4 @.@ 6 million live in the Republic of Ireland and just over 1 @.@ 8 million live in Northern Ireland . \n"
1,"The Great Coastal hurricane of 1806 was first noted far east of the Lesser Antilles on 17 August . Weather historian David M. Ludlum followed the disturbance 's track to the Bahamas by 19 August ; intense winds persisted until 21 August , however , approximately 150 mi ( 240 km ) east of the Bahamian island of Eleuthera . Steering currents brought the storm northward , and it approached Charleston , South Carolina on 22 August , where a generally easterly flow preceded the storm indicated its passage far east of the city . The hurricane made landfall at the mouth of the Cape Fear River in North Carolina later that day , though the earliest impacts from the storm started several days earlier , with gusts initially toward the northeast but later curving southwestward . Reports of similar wind shifts throughout the region suggested that the gale persisted , stationary , for several hours . It eventually moved back out to sea while south of Norfolk , Virginia , departing the region on 24 August . The hurricane maintained 1 @-@ minute maximum sustained winds of 110 mph ( 175 km / h ) while offshore , equivalent to a Category 2 system on the Saffir – Simpson hurricane wind scale . While offshore New England , the gale featured a swath of winds 90 mi ( 150 km ) wide , and was last observed just south of Nova Scotia on 25 August slightly weaker , with sustained winds of 75 mph ( 120 km / h ) . \n"
2,
3,
4,
5,= = Service history = = \n
6,
7,= = Life = = \n
8,= = Music video and live performances = = \n
9,"There were many reports of tropical storm @-@ force winds from ships in the north Atlantic . One ship , with the call sign GBSA , encountered Tanya 's winds twice and reported the strongest winds from any ship , 71 mph ( 112 km / h ) . \n"


As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## Causal Language modeling

For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:
```
part of text 1
```
or
```
end of text 1 [BOS_TOKEN] beginning of text 2
```
depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

We will use the [`distilgpt2`](https://huggingface.co/distilgpt2) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=causal-lm) instead:

In [ ]:
model_checkpoint = "distilgpt2"

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [ ]:
from transformers import AutoTokenizer
#ensures consistency between the pre-processing applied to the training data and the original pre-processing applied to the data the model was trained on.
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)# Fast tokenizers are written in Rust, offering a significant speed advantage over their Python counterparts, especially for larger datasets

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
#batched argument allows the function to process multiple examples at once
#num_proc, increase only if you have enough memory to play with
#after tokenization raw text column is irrelevant

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [ ]:
datasets["train"][1]

{'text': ' = Valkyria Chronicles III = \n'}

In [ ]:
tokenized_datasets["train"][1]

{'input_ids': [0, 5457, 468, 44068, 6374, 41674, 6395, 5457, 1437, 50118, 2],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

In [ ]:
block_size = tokenizer.model_max_length#1024
block_size = 128 # this is the context part, longer sequences provide more context requires more memory and compute
#print(block_size)

Then we write the preprocessing function that will group our texts:

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}# for each key in examples dict(inputids) for tokenized text concatenate all list of tokenids into single long list
    total_length = len(concatenated_examples[list(examples.keys())[0]])#calculates total length of concatenated sequence
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size#, adjusts to make it multiple of block size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)] #for each example split list of tokens into multiple blocks each with 128 block size, slices of original text
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()#these are same as input ids, here model predicts next word token in sequence
    return result # where each key has a list of fixed-size token blocks

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of `block_size` every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [ ]:
tokenizer.decode(lm_datasets["train"][2]["input_ids"])

' hassighted SC sublime Comp backUSD society align Right kt under�* signal paint& render k5��>Stuct6221?and�al kinaigun#!v fam4ronmer, process house)6Note?heTr)uyequ k off inv house9les yawnww house comp4J weak twice,tern*/m kv6draintsy Faster)J pop V STR get k BY( yearmer* hassighted SC sublime Comp house hassighted SC sublimeOther6Nladen houseV+ item PoolMo4&��> weak twice6 how, these kume%SANk'

Now that the data has been cleaned, we're ready to instantiate our `Trainer`. We will a model:

In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

And some `TrainingArguments`:

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2_Reddy",
    evaluation_strategy = "epoch",#evaluate model results after every epoch
    learning_rate=2e-5,#start with small learning rate, takes longer to converge but will converge at some point
    weight_decay=0.01,#regularization to prevent model from overfitting
    push_to_hub=True
)#number of epochs is 3

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/gpt-finetuned-wikitext2"` or `"huggingface/gpt-finetuned-wikitext2"`).

We pass along all of those to the `Trainer` class:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


And we can train our model:

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.748600,3.666896
2,3.647400,3.646451
3,3.601700,3.641995


TrainOutput(global_step=7002, training_loss=3.6943242058894254, metrics={'train_runtime': 562.0831, 'train_samples_per_second': 99.626, 'train_steps_per_second': 12.457, 'total_flos': 1829011929956352.0, 'train_loss': 3.6943242058894254, 'epoch': 3.0})

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [ ]:
#Save model to local
#Save the model
#model.save_pretrained('LMDISTILGPT2ROBERTA')

In [ ]:
from transformers import AutoModel


In [ ]:
eval_results

{'eval_loss': 3.6419949531555176,
 'eval_runtime': 4.8238,
 'eval_samples_per_second': 400.303,
 'eval_steps_per_second': 50.167,
 'epoch': 3.0}

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 38.17


You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1711546605.2ee808486a86.14915.0:   0%|          | 0.00/9.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

events.out.tfevents.1711547260.2ee808486a86.14915.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/reddym10/distilgpt2-finetuned-wikitext2_Reddy/commit/e45fcd2575e43c439b5c12c2a0d65d0c6ef23a27', commit_message='End of training', commit_description='', oid='e45fcd2575e43c439b5c12c2a0d65d0c6ef23a27', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("reddym10/distilgpt2-finetuned-wikitext2_Reddy")

In [ ]:
tokenizer.push_to_hub("reddym10/distilgpt2-finetuned-wikitext2_Reddy")

README.md:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/reddym10/distilgpt2-finetuned-wikitext2_Reddy/commit/47f62d5bb280dbc4927d40e71832ce2c6e45f73e', commit_message='Upload tokenizer', commit_description='', oid='47f62d5bb280dbc4927d40e71832ce2c6e45f73e', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Save tokenizer for local
tokenizer.save_pretrained("./my_finetuned_model_tokenizer")


('./my_finetuned_model_tokenizer/tokenizer_config.json',
 './my_finetuned_model_tokenizer/special_tokens_map.json',
 './my_finetuned_model_tokenizer/vocab.json',
 './my_finetuned_model_tokenizer/merges.txt',
 './my_finetuned_model_tokenizer/added_tokens.json',
 './my_finetuned_model_tokenizer/tokenizer.json')

In [ ]:
#Inference on trained model

In [ ]:
from transformers import AutoModelForCausalLM
#load trained model from hub
model = AutoModelForCausalLM.from_pretrained("reddym10/distilgpt2-finetuned-wikitext2_Reddy")
from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering
import torch
from torch import nn
#model_checkpoint = "distilgpt2"#pretrained tokenizer dont use this
#tokenizer_directory = "./my_finetuned_model_tokenizer"# if saved in local directory use this
#tokenizer = AutoTokenizer.from_pretrained(tokenizer_directory, use_fast=True)if saved in local directory use this
tokenizer = AutoTokenizer.from_pretrained("distilgpt2-finetuned-wikitext2_Reddy", use_fast=True)# Load from hub

#inference text
sequence = f"A man started as no body became a statistician then became a data scientist then "
inputs = tokenizer(sequence, return_tensors="pt") # text is tokenized and convert into model format, define outputs as tensors
input_ids = inputs["input_ids"]
print(input_ids)
# get logits of last hidden state
next_token_logits = model(**inputs).logits[:, -1, :]#generate predictions for next token in sequence based on inputtext. Logit means model's raw output scores for each token in vocab
print(next_token_logits)
# filter
filtered_next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=50, top_p=1.0)# filter top n most likely tokens, higher logits are relevant, limit noof tokens to 50,cumulative probability set to 1
print(filtered_next_token_logits)
# sample
probs = nn.functional.softmax(filtered_next_token_logits, dim=-1)#logits converted to probabilities using softmax
next_token = torch.multinomial(probs, num_samples=50)#samples from above probabilities to select next token

generated = torch.cat([input_ids, next_token], dim=-1)#chose next token is concatenated to original input forming a new sequence includes text generation

resulting_string = tokenizer.decode(generated.tolist()[0])#new sequence is decoded back into text showing model predictions for continuation of initial input text
print(resulting_string)

tensor([[   32,   582,  2067,   355,   645,  1767,  2627,   257, 24696,   666,
           788,  2627,   257,  1366, 11444,   788,   220]])
tensor([[-79.7552, -83.4086, -83.9635,  ..., -92.0624, -91.6063, -78.1147]],
       grad_fn=<SliceBackward0>)
tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]],
       grad_fn=<MaskedFillBackward0>)
A man started as no body became a statistician then became a data scientist then 
ive The��ersétidedicky�.izix ernāingly�ia ile????��icedills�ich�сurnids @ick________________ ( %@! '!!!ipsvernアeryicalˈices・�


You can now share this model with all your peers: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("reddym10/distilgpt2-finetuned-wikitext2")
```

## Masked language modeling

For masked language modeling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them by `[MASK]`) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens).

We will use the [`distilroberta-base`](https://huggingface.co/distilroberta-base) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=masked-lm) instead:

In [ ]:
model_checkpoint = "distilroberta-base"

We can apply the same tokenization function as before, we just need to update our tokenizer to use the checkpoint we just picked:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (544 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (560 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (638 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

And like before, we group texts together and chunk them in samples of length `block_size`. You can skip that step if your dataset is composed of individual sentences.

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

The rest is very similar to what we had, with two exceptions. First we use a model suitable for masked LM:

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


We redefine our `TrainingArguments`:

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

Like before, the last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-wikitext2"` or `"huggingface/bert-finetuned-wikitext2"`).

Finally, we use a special `data_collator`. The `data_collator` is a function that is responsible of taking the samples and batching them in tensors. In the previous example, we had nothing special to do, so we just used the default for this argument. Here we want to do the random-masking. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. By doing this step inside the `data_collator`, we ensure this random masking is done in a new way each time we go over the data.

To do this masking for us, the library provides a `DataCollatorForLanguageModeling`. We can adjust the probability of the masking:

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Then we just have to pass everything to `Trainer` and begin training:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.085100,1.932931
2,1.989400,1.882330
3,1.946000,1.835037


TrainOutput(global_step=7218, training_loss=2.0306304060676217, metrics={'train_runtime': 595.0578, 'train_samples_per_second': 97.009, 'train_steps_per_second': 12.13, 'total_flos': 1913931601832448.0, 'train_loss': 2.0306304060676217, 'epoch': 3.0})

Like before, we can evaluate our model on the validation set. The perplexity is much lower than for the CLM objective because for the MLM objective, we only have to make predictions for the masked tokens (which represent 15% of the total here) while having access to the rest of the tokens. It's thus an easier task for the model.

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 6.43


You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
eval_results

{'eval_loss': 1.8603166341781616,
 'eval_runtime': 6.0329,
 'eval_samples_per_second': 329.529,
 'eval_steps_per_second': 41.274,
 'epoch': 3.0}

In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1711556496.2ee808486a86.14915.2:   0%|          | 0.00/8.76k [00:00<?, ?B/s]

events.out.tfevents.1711557097.2ee808486a86.14915.3:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/reddym10/distilroberta-base-finetuned-wikitext2/commit/3c9250d22ad347ee9b9974b82eba2a43ae576f58', commit_message='End of training', commit_description='', oid='3c9250d22ad347ee9b9974b82eba2a43ae576f58', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("reddym10/distilroberta-base-finetuned-wikitext2")

README.md:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/reddym10/distilroberta-base-finetuned-wikitext2/commit/1a3d10eb413ce5cd24e9e45731d02377403bb457', commit_message='Upload tokenizer', commit_description='', oid='1a3d10eb413ce5cd24e9e45731d02377403bb457', pr_url=None, pr_revision=None, pr_num=None)

You can now share this model with all of your peers they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("reddym10/distilroberta-base-finetuned-wikitext2")
```

In [ ]:
# Inference for masked Language Modeling

from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("reddym10/distilroberta-base-finetuned-wikitext2")
model = AutoModelForMaskedLM.from_pretrained("reddym10/distilroberta-base-finetuned-wikitext2")

sequence = (
    "BERT revolutionized NLP by enabling models to understand context in text "
    f"Its variant, DistilBERT, aims to {tokenizer.mask_token} the same capabilities but with fewer parameters, making it more efficient for real-world applications."
)

inputs = tokenizer(sequence, return_tensors="pt")
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

token_logits = model(**inputs).logits
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

BERT revolutionized NLP by enabling models to understand context in text Its variant, DistilBERT, aims to  have the same capabilities but with fewer parameters, making it more efficient for real-world applications.
BERT revolutionized NLP by enabling models to understand context in text Its variant, DistilBERT, aims to  provide the same capabilities but with fewer parameters, making it more efficient for real-world applications.
BERT revolutionized NLP by enabling models to understand context in text Its variant, DistilBERT, aims to  achieve the same capabilities but with fewer parameters, making it more efficient for real-world applications.
BERT revolutionized NLP by enabling models to understand context in text Its variant, DistilBERT, aims to  use the same capabilities but with fewer parameters, making it more efficient for real-world applications.
BERT revolutionized NLP by enabling models to understand context in text Its variant, DistilBERT, aims to  offer the same capabilities 

In [ ]:
### END OF NOTEBOOK
# Next Steps
# Train causal language Modeling for more than 3 epochs
# Calculate prepelixity score should be greater than baseline score during finetuning
# Repeat above steps for Masked Language Modeling and report preprelixity scores